In [7]:
import import_ipynb
import collections
import numpy as np
import torch
import time
import os
import torch.optim as optim
from torchvision import transforms
from model import  PyramidFeatures,RegressionModel,ClassificationModel,ResNet, model_resnet18,  model_resnet34,  model_resnet50,  model_resnet101,  model_resnet152
from dataloader import CocoDataset, Coco_testdata, collater_test,collater, Resizer, AspectRatioBasedSampler, Augmenter, Normalizer
from torch.utils.data import Dataset, DataLoader
from coco_eval import evaluate_coco #coco eval의 코드실행
import sys
print(sys.version)
print(torch.version.cuda)
print(torch.__version__)
from pycocotools.coco import COCO

3.7.15 (default, Nov 24 2022, 18:44:54) [MSC v.1916 64 bit (AMD64)]
11.6
1.13.1+cu116


In [2]:
print('CUDA available: {}'.format(torch.cuda.is_available())) #CUDA available
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)# cuda 사용가능 확인
print(torch.cuda.device_count()) #사용 가능 GPU 개수

CUDA available: True
cuda
1


In [3]:
# 데이터 저장 장소
save_path='C:/Users/user/Desktop/coco_flexing'

In [5]:
def Retina(path,depth,epochs):
    print(f'resnet_{depth} backbone 사용하고, {epochs}번 반복하여 학습합니다')
    #train coco dataset
    dataset_train=CocoDataset(data_dir=path+'/'+'train',set_name='train',transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
    dataset_val=CocoDataset(data_dir=path+'/'+'valid',set_name='val',transform=transforms.Compose([Normalizer(), Resizer()]))
    print('train coco build, valid coco bild')    
        
    #sampler->batch size로 데이터를 나눠주는듯 함
    sampler=AspectRatioBasedSampler(dataset_train, batch_size=4, drop_last=False)
    dataloader_train = DataLoader(dataset_train, num_workers=0, collate_fn=collater, batch_sampler=sampler) #num_workers=3이 default
    
    #dataloader_train=DataLoader(dataset_train,num_workers=1,collate_fn=collater,batch_size=2)
    if dataset_val is not None:
        sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=4, drop_last=False) #ex) [756,777]
        dataloader_val = DataLoader(dataset_val, num_workers=0, collate_fn=collater, batch_sampler=sampler_val)
    print('dataloader build success')
    print('총 클래스 개수',dataset_train.num_classes())
    print('총 데이터',len(dataset_train))
    print('dataloader iteration',len(dataloader_train))
    # Create the model depth에 따라 모델 선택 달라짐
    if depth == 18:
        retinanet = model_resnet18(num_classes=dataset_train.num_classes(), pretrained=False)
    elif depth == 34:
        retinanet = model_resnet34(num_classes=dataset_train.num_classes(), pretrained=False)
    elif depth == 50:
        retinanet = model_resnet50(num_classes=dataset_train.num_classes(), pretrained=True) #현재 오류 발생 지점
    elif depth == 101:
        retinanet = model_resnet101(num_classes=dataset_train.num_classes(), pretrained=False)
    elif depth == 152:
        retinanet = model_resnet152(num_classes=dataset_train.num_classes(), pretrained=False)
    else:
        raise ValueError('Unsupported model depth, must be one of 18, 34, 50, 101, 152')
        
    #모델 파라미터 사이즈 확인 코드
    # for name, param in retinanet.named_parameters():
    #     print(name, param.size())
        
    use_gpu = True #GPU 사용여부 True
    GPU_num=torch.cuda.device_count()
    #print(use_gpu)
    
    #GPU한개일때
#    if use_gpu:
#        print('GPU사용')
#        if torch.cuda.is_available():
#            print('CUDA사용')
#            if GPU_num==1:
#                print('사용 GPU 1개')
#                #retinanet = retinanet.cuda()
#                retinanet=retinanet.to(device) #cuda로 이동
    retinanet=retinanet.to(device)
                
    #GPU 한개 이상
#     if use_gpu:
#         if torch.cuda.is_available():
#             if GPU_num > 1:
#                 print("GPU 1개 이상 사용")
#                 retinanet = torch.nn.DataParallel(retinanet).to(device)
                
#     else:
#         print('CPU 사용')
#         retinanet=torch.nn.DataParallel(retinanet).to(device)
                
                

#     if use_gpu:
#         if torch.cuda.is_available(): #cuda 사용여부
#             retinanet = retinanet.cuda() #cuda memory에 RetinaNet load

#     if torch.cuda.is_available():
#         print('CUDA사용')
#         retinanet = torch.nn.DataParallel(retinanet).cuda() #병렬처리->코드 알아보기
#     else:
#         print('CPU사용')
#         retinanet = torch.nn.DataParallel(retinanet)

    retinanet.training = True #train여부

    optimizer = optim.Adam(retinanet.parameters(), lr=1e-5) #Adam optimizer사용

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',patience=3, verbose=True) #scheduler 설정

    loss_hist = collections.deque(maxlen=500)

    retinanet.train() #train Mode
    if GPU_num==1:
        retinanet.freeze_bn() #batchnormalization 고정
    else:
        # retinanet.module.freeze_bn() #병렬처리할때 ON
        pass

    print('Num training images: {}'.format(len(dataset_train))) #Num training images 57~~~

    for epoch_num in range(epochs):#학습 시작
        print("Train start")
        print(f'epoch {epoch_num+1} start')

        retinanet.train() #train Mode
        if GPU_num==1:
            retinanet.freeze_bn()
        else:
            # retinanet.module.freeze_bn()
            pass

        epoch_loss = [] #loss append list
        print('Model학습 시작')
        for iter_num, data in enumerate(dataloader_train): #index, data->image=2,3,608,640 for문 제대로안돌아감
            optimizer.zero_grad() #gradient 초기화
            #print('Model학습 시작')
            classification_loss, regression_loss = retinanet([data['img'].to(device).float(), data['annot'].to(device)]) #classification, regression loss,여기서부터 오류발생
            classification_loss = classification_loss.mean() #classification loss mean
            regression_loss = regression_loss.mean() #regression loss mean

            loss = classification_loss + regression_loss #두 loss sum

            loss.backward() #back propagation
            torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1) #gradient전체 norm이 0.1을 넘지 않도록 방지
            optimizer.step() #가중치 업로드
            loss_hist.append(float(loss)) #loss append
            epoch_loss.append(float(loss)) #epoch당 loss

            #epoch 마다 loss값을 츨력해줌
            # if iter_num%50==0:
            #     print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(epoch_num+1, iter_num+1, float(classification_loss), float(regression_loss), np.mean(loss_hist)))

            del classification_loss #print 끝나면 classification_loss삭제
            del regression_loss #print 끝나면 regression loss삭제함.
        print(f'{epoch_num+1} 학습 완료')
        train_yes=epoch_num+1
        if train_yes==epoch_num+1:
            #evaluate validation dataset
            print("Start Evaluating val datset")
            #evaluate_coco(dataset_val,retinanet)
            class_AP=evaluate_coco(dataset_val,retinanet,path=path) #map, ap 저장
            #전체 결과 출력 한 epoch당임
            # 전체 클래스 mAP50
            print('-'*40)
            print(f'전체 클래스 mAP:50={round(class_AP.stats[1]*100,2)}%')
            print('-'*40)
            print(f'Large 객체에 대한 mAP:50:0.95={round(class_AP.stats[5]*100,2)}%')
            print('-'*40)
            print(f'Medium 객체에 대한 mAP:50:0.95={round(class_AP.stats[4]*100,2)}%')
            print('-'*40)
            print(f'Small 객체에 대한 mAP:50:0.95={round(class_AP.stats[3]*100,2)}%')
            print('-'*40)
        else:
            pass
        
        scheduler.step(np.mean(epoch_loss))
    print('Train, Evaluation 완료, 결과값 저장 중')
    if GPU_num>1:
        torch.save(retinanet.module, './models/retina_final.pt')
        torch.save(retinanet.module.state_dict(), f'./models/retina_stat.pt')
        
    else:
        torch.save(retinanet,'./models/retina_final.pt')
        torch.save(retinanet.state_dict(), f'./models/retina_stat.pt')
    print('Model 저장 및 학습 및 검증 종료')
    retinanet.eval()
    return class_AP #mAP, AP저장

In [6]:
start_time = time.time()

class_AP =Retina(path=save_path,depth=50,epochs=10)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time} seconds")

resnet_50 backbone 사용하고, 10번 반복하여 학습합니다
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
train coco build, valid coco bild
dataloader build success
총 클래스 개수 2
총 데이터 80
dataloader iteration 20
Num training images: 80
Train start
epoch 1 start
Model학습 시작
1 학습 완료
Start Evaluating val datset
Loading and preparing results...
DONE (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.01s).
2
category0_Background : -1.0
category1_halibut : 0.06243286233453326
(all categories) mAP : 0.03121643116726663
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.062
2
category0_Background : -1.0
category1_halibut : 0.18358556559657277
(all categories) mAP : 0.09179278279

In [9]:
def Model_test(path):
    dataset_test = CocoDataset(data_dir=save_path+'/'+'test', set_name='test', transform=transforms.Compose([Normalizer(), Resizer()]))
    #dataset_test=Coco_testdata(data_dir=save_path+'/'+'test', set_name='test', transform=transforms.Compose([Normalizer(), Resizer()])) #COCO형식으로 데이터 불러오기
    # Create the model
    retinanet_test = model_resnet50(num_classes=dataset_test.num_classes(), pretrained=True)

    use_gpu = True

    if use_gpu:
        if torch.cuda.is_available():
            retinanet_test = retinanet_test.to(device)

    if torch.cuda.is_available():
        # stat_dict=torch.load('./model_weights/model_stat.pt',map_location=device)
        # from collections import OrderedDict
        # new_stat_dict=OrderedDict()
        # for k, v in stat_dict.items():
        #     name=k[7:]
        #     new_stat_dict[name] = v
        # retinanet.load_state_dict(new_stat_dict)

        retinanet_test=torch.load('C:/Users/user/Desktop/project/Retina/models/retina_final.pt')
        retinanet_test = retinanet_test.to(device)
    else:
        #retinanet.load_state_dict(torch.load('C:/Users/user/Desktop/project/Retina/models/retina_stat.pt'),map_location=device)
        retinanet_test = torch.load('C:/Users/user/Desktop/project/Retina/models/retina_final.pt')

    retinanet_test.training = False
    retinanet_test.eval()
    retinanet_test.freeze_bn()

    evaluate_coco(dataset_test, retinanet_test,path=path)

In [10]:
Model_test(path=save_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.00s).
2
category0_Background : -1.0
category1_halibut : 0.6422587313676422
(all categories) mAP : 0.3211293656838211
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.642
2
category0_Background : -1.0
category1_halibut : 0.9297029702970298
(all categories) mAP : 0.4648514851485149
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=300 ] = 0.930
2
category0_Background : -1.0
category1_halibut : 0.8148514851485149
(all categories) mAP : 0.40742574257425745
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=300 ] = 0.815
 Average Precision  (AP) @[ IoU=0.50:0.95 | area

In [11]:
import os
import copy
import time
import copy
import pdb
import matplotlib.pyplot as plt
import sys
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from dataloader import UnNormalizer
from collections import Counter
import pandas as pd
import warnings
import csv

warnings.filterwarnings("ignore")

In [12]:
class_name=['filename','Halibut']
counter_df=pd.DataFrame(columns=class_name)

In [13]:
print(counter_df)

Empty DataFrame
Columns: [filename, Halibut]
Index: []


# Visual save 코드 수정하기(06-07일까지)

In [40]:
def visual_save(path):
    class_name=['filename','Halibut'] #컬럼명 설정
    counter_df=pd.DataFrame(columns=class_name) #데이터 프레임 만들기
    
    # 데이터 Load
    dataset_test=Coco_testdata(data_dir=path+'/'+'test', set_name='test', transform=transforms.Compose([Normalizer(), Resizer()])) #COCO형식으로 데이터 불러오기, test dataset형식 따로 있음
    sampler_test=AspectRatioBasedSampler(dataset_test, batch_size=1, drop_last=False) #batch size는 1로 설정해야 1장씩 탐지결과 출력가능
    dataloader_test = DataLoader(dataset_test, num_workers=0, collate_fn=collater_test, batch_sampler=sampler_test) #dataloader
    retinanet=torch.load('C:/Users/user/Desktop/project/Retina/models/retina_final.pt') #model load
    use_gpu=True #GPU 사용 설정
    
    if use_gpu:
        if torch.cuda.is_available():
            retinanet=retinanet.cuda() #Model GPU에 올리기
            
    else:
        retinanet=retinanet.to(device)
        
    # GPU여러개일때 실행
    # if torch.cuda.is_available():
    #     retinanet = torch.nn.DataParallel(retinanet).cuda()
    # else:
    #     retinanet = torch.nn.DataParallel(retinanet)
    
    retinanet.eval() #Model Train Mode
    unnormalize=UnNormalizer() #이미지 정규화하는 코드 불러오기(Dataloader에 class 불러오기)
    
    #바운딩박스 적용
    def draw_caption(image,box,caption):
        b=np.array(box).astype(int)
        cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
        cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
        
    print('Testdata Model적용 중....')   
    
    # 배치 사이즈별 모델 적용
    for idx, data in enumerate(dataloader_test):
        filename=data['filename'] # 배치 당 filename 저장
        with torch.no_grad(): #test 버전은 backpropagation 진행 X
            st = time.time()
            if torch.cuda.is_available():
                scores, classification, transformed_anchors = retinanet(data['img'].cuda().float()) #scores, classification, transformed_anchors 객체 저장
            else:
                scores, classification, transformed_anchors = retinanet(data['img'].float())
            #scores 식별된 객체들의 Iou값 반환 0.5이상값만 탐지 성공으로 간주함
            #classification->label의 인덱스값 추출
            #transformed_anchors->scores 0.5이상의 값들의 객체에 대한 좌표값만을 출력해줌
            #print('Elapsed time: {}'.format(time.time()-st))
            #print('Testdata Model적용 중....')
            idxs = np.where(scores.cpu()>0.5)
            img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()
            img[img<0] = 0
            img[img>255] = 255
            

            img = np.transpose(img, (1, 2, 0))
            img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)
            label_name_img=[]
            for j in range(idxs[0].shape[0]):
                bbox = transformed_anchors[idxs[0][j], :] #예측된 x,y좌표 출력 
                x1 = int(bbox[0])
                y1 = int(bbox[1])
                x2 = int(bbox[2])
                y2 = int(bbox[3])
                label_name = dataset_test.labels[int(classification[idxs[0][j]])] #label name 출력
                label_name_img.append(label_name) #label명 따로 뽑기 어떤 객체들이 식별되었는지 저장 각 이미지 당
                draw_caption(img, (x1, y1, x2, y2), label_name) #식별된 객체들 이미지상에 표현함
                cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2) #박스 생성
                #print(label_name)
            # cv2.imshow('img', img)
            # cv2.waitKey(0)
            #각 이미지당 나온 클래스값 빈도수 세기
            counter=Counter(label_name_img) #label_name_img ex)['PU_NA', 'PU_NA', 'PU_NA', 'PU_NA'] 이런식으로 식별된 객체가 나옴
            counter_df.loc[idx,'filename']=filename
            for key,value in counter.items():
                counter_df.loc[idx,key]=int(value)
                
            #cv2.imwrite(f'C:/Users/user/Desktop/project/Retina/images/{filename}.jpg',img) #전체 이미지 탐지 결과 저장
            # 질병에 걸린 이미지들만 따로 분류하여 저장
            #현재 disease list는 AA_NA, disease 두 종류=>프로젝트를 진행하며 확정할 예정임
            if 'SA' in label_name_img:
                cv2.imwrite(f'C:/Users/user/Desktop/project/Retina/disease_image/{filename}.jpg',img)
                #print(f'꿀벌의 질병이 식별된 이미지는 C:/Users/user/Desktop/project/Retina/disease_image 폴더에서 확인하세요')
            else: #정상이미지 저장
                 cv2.imwrite(f'C:/Users/user/Desktop/project/Retina/images/{filename}.jpg',img)          
    #counter df NaN->0으로 대치
    counter_df.fillna(0,inplace=True)
    counter_df['filename']=counter_df['filename'].astype(str)
    counter_df['filename'] = counter_df['filename'].str.replace('[', '').str.replace(']', '')
    for i in range(len(counter_df)):
        if counter_df['Halibut'][i]>0:
            counter_df.loc[i,'disease_presence']=1 #질병이 있는 이미지에는 1
        else:
            counter_df.loc[i,'disease_presence']=0 #질병이 없는 이미지에는 0
    counter_df.to_csv('C:/Users/user/Desktop/project/Retina/detection_list/detection_list.csv',index=False, encoding='cp949')
    return counter_df

In [41]:
counter_df=visual_save(path=save_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Testdata Model적용 중....
